In [1]:
#!pip install shap --quiet

In [1]:
from cashe import get_train_test_split, get_training_inputs, drop_nomodel_columns
from src.model_app import ModelApplication, ModelApplicationDeployment
from src.utilities_rejection import RejectionReasonLabeling
from src.data_local import MergedDataPreprocessing
from src.lstm_encoder import LSTMEmbedding
import pandas as pd
import numpy as np

In [2]:
df_train, df_test = get_train_test_split('data/HJH/13-06-2024')

df_train = df_train[df_train['OUTCOME'] !='APPROVED']
df_test = df_test[df_test['OUTCOME'] !='APPROVED']

In [3]:
df_train.columns

Index(['REQUEST_DATE', 'VISIT_NO', 'VISIT_DATE', 'EMERGENCY_INDICATOR',
       'PROVIDER_DEPARTMENT_CODE', 'PROVIDER_DEPARTMENT',
       'DOCTOR_SPECIALTY_CODE', 'DOCTOR_CODE', 'PATIENT_NO', 'EPISODE_NO',
       ...
       'SEQUENCE_NO', 'UNIT_PRICE_STOCKED_UOM', 'UNIT_PRICE_NET',
       'DISCOUNT_PERCENTAGE', 'NOTES', 'OUTCOME', 'APPROVED_QUNATITY',
       'Episode_Key', 'ICD10', 'Diagnosis_Key'],
      dtype='object', length=101)

In [4]:
train_rej = RejectionReasonLabeling(df_train)
train_rej.recoginze_label()

train_rej = RejectionReasonLabeling(df_test)
train_rej.recoginze_label()

In [5]:
df_train['NPHIES_LABEL'] = np.where(df_train['NPHIES_CODE'] == 'MN-1-1', 1, 0)
df_test['NPHIES_LABEL'] = np.where(df_test['NPHIES_CODE'] == 'MN-1-1', 1, 0)

In [7]:
df_train[['ICD10','NOTES','NPHIES_LABEL','NPHIES_CODE']]

,ICD10,NOTES,NPHIES_LABEL,NPHIES_CODE
398449,K02,Rejected for the reason Sorry you are not an a...,0,CV-1-1
398453,N80.1,Rejected for the reason Sorry you are not an a...,0,CV-1-1
398454,N80.1,Rejected for the reason Sorry you are not an a...,0,CV-1-1
398450,N80.1,Rejected for the reason Sorry you are not an a...,0,CV-1-1
398452,N80.1,Rejected for the reason Sorry you are not an a...,0,CV-1-1
...,...,...,...,...
316179,J18.0,MN-1-1,1,MN-1-1
301017,I21,BE-1-1,0,BE-1-1
301022,I21,BE-1-1,0,BE-1-1
301021,I21,BE-1-1,0,BE-1-1


In [8]:
X_train, y_train, X_test, y_test = get_training_inputs(df_train, df_test)
X_train = drop_nomodel_columns(X_train);  X_test = drop_nomodel_columns(X_test)
X_train.drop(columns=['NPHIES_CODE'],inplace=True); X_test.drop(columns=['NPHIES_CODE'],inplace=True)

In [9]:
X_train['ICD10']

398449      K02
398453    N80.1
398454    N80.1
398450    N80.1
398452    N80.1
          ...  
316179    J18.0
301017      I21
301022      I21
301021      I21
301023      I21
Name: ICD10, Length: 76988, dtype: object

In [10]:
preprocessing_train = MergedDataPreprocessing(X_train)
X_train_prep = preprocessing_train.columns_prep(service_encoding=False)

preprocessing_test = MergedDataPreprocessing(X_test)
X_test_prep = preprocessing_test.columns_prep(service_encoding=False)
lstm_embedding = LSTMEmbedding()

X_train_encoded = preprocessing_train.column_embedding(X_train_prep, is_service=True)
X_test_encoded = preprocessing_train.column_embedding(X_test_prep, is_service=True)

E:\Projects\Claims_Rejection\src\data_local.py:276: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[column]= df[column].replace(column_encoding)
E:\Projects\Claims_Rejection\src\data_local.py:284: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['PatientAgeRange']= df['PatientAgeRange'].replace(age_encoding)
E:\Projects\Claims_Rejection\src\data_local.py:276: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(cop

In [11]:
X_train_encoded.ICD10

398449                                                  K02
398453                                                N80.1
398454                                                N80.1
398450                                                N80.1
398452                                                N80.1
                                ...                        
316179    Pneumonia, unspecified organism,Bronchopneumon...
301017                                                  I21
301022                                                  I21
301021                                                  I21
301023                                                  I21
Name: ICD10, Length: 76988, dtype: object

In [12]:
X_train_encoded = preprocessing_train.column_embedding(X_train_encoded,is_service=False)
X_test_encoded = preprocessing_train.column_embedding(X_test_encoded,is_service=False)
X_train_encoded

,EMERGENCY_INDICATOR,PROVIDER_DEPARTMENT_CODE,DOCTOR_SPECIALTY_CODE,DOCTOR_CODE,PATIENT_AGE,UNIT_OF_AGE,PATIENT_NATIONALITY,PATIENT_MARITAL_STATUS,PATIENT_GENDER,CLAIM_TYPE,...,ICDText7,ICDText8,ICDText9,ICDText10,ICDText11,ICDText12,ICDText13,ICDText14,ICDText15,ICDText16
398449,0,4,22.00,7352,0,2,2,0,0,1,...,0,0,0,0,0,0,0,0,0,0
398453,0,25,18.06,9638,33,1,2,1,0,1,...,0,0,0,0,0,0,0,0,0,0
398454,0,25,18.06,9638,33,1,2,1,0,1,...,0,0,0,0,0,0,0,0,0,0
398450,0,25,18.06,9638,33,1,2,1,0,1,...,0,0,0,0,0,0,0,0,0,0
398452,0,25,18.06,9638,33,1,2,1,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316179,0,22,14.00,5096,13,1,2,0,1,1,...,0,0,0,0,0,0,0,0,0,0
301017,0,29,8.02,3396,45,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
301022,0,29,8.02,3396,45,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
301021,0,29,8.02,3396,45,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [13]:
model_rejection = ModelApplication(X_train=X_train_encoded, y_train=y_train['NPHIES_LABEL'],
                                   X_test=X_test_encoded, y_test=y_test['NPHIES_LABEL'])

In [17]:
model_rejection.get_model(force_retrain=False,model_type='rejection')

XGBoost Model is loading from disk..


In [15]:
## runtime 1
model_rejection.evaluate_model()

{'XGBoost': {'Accuracy': 0.89,
  'Precision': 0.74,
  'Recall': 0.58,
  'F1 Score': 0.65}}

In [16]:
## runtime 2
model_rejection.evaluate_model()

{'XGBoost': {'Accuracy': 0.89,
  'Precision': 0.74,
  'Recall': 0.58,
  'F1 Score': 0.65}}

In [23]:
model_rejection_deployed = ModelApplicationDeployment(X_test)
model_rejection_deployed.interprete()

XGBoost Model is loading from disk..


,main_contributing_feature
0,PURCHASER_CODE
1,PURCHASER_CODE
2,PURCHASER_CODE
3,PURCHASER_CODE
4,PURCHASER_CODE
...,...
19811,CONTRACT_NO
19812,PURCHASER_CODE
19813,PURCHASER_CODE
19814,PURCHASER_CODE


In [24]:
X_train.columns


Index(['NET_WITH_VAT', 'QTY', 'LINE_CLAIMED_AMOUNT_SAR', 'CO_INSURANCE',
       'LINE_ITEM_DISCOUNT', 'NET_VAT_AMOUNT', 'PATIENT_VAT_AMOUNT',
       'VAT_PERCENTAGE', 'TREATMENT_TYPE_INDICATOR', 'DURATION',
       'QTY_STOCKED_UOM', 'UNIT_PRICE_STOCKED_UOM', 'UNIT_PRICE_NET',
       'DISCOUNT_PERCENTAGE', 'EMERGENCY_INDICATOR',
       'PROVIDER_DEPARTMENT_CODE', 'DOCTOR_SPECIALTY_CODE', 'DOCTOR_CODE',
       'PATIENT_AGE', 'UNIT_OF_AGE', 'PATIENT_NATIONALITY',
       'PATIENT_MARITAL_STATUS', 'PATIENT_GENDER', 'CLAIM_TYPE',
       'TOTAL_CLAIMED_AMOUNT_SAR', 'TOTAL_DISCOUNT', 'TOTAL_DEDUCTIBLE',
       'TOTAL_PATIENT_VATAMOUNT', 'DEPARTMENT_TYPE', 'TREATMENT_TYPE',
       'PURCHASER_CODE', 'CONTRACT_NO', 'NEW_BORN', 'PatientAgeRange',
       'CombinedText1', 'CombinedText2', 'CombinedText3', 'CombinedText4',
       'CombinedText5', 'CombinedText6', 'CombinedText7', 'CombinedText8',
       'CombinedText9', 'CombinedText10', 'CombinedText11', 'CombinedText12',
       'CombinedText13', 'C

## Models Comparison

In [23]:
from src.model_train import ModelTrainer

X_train_encoded.fillna(0,inplace=True)
X_test_encoded.fillna(0,inplace=True)

model_runner = ModelTrainer(X_train=X_train_encoded,y_train=y_train['NPHIES_LABEL'],
                            X_test=X_test_encoded,y_test=y_test['NPHIES_LABEL'])

model_runner.train_models()

[LightGBM] [Info] Number of positive: 17007, number of negative: 62680
[LightGBM] [Info] Total Bins 4704
[LightGBM] [Info] Number of data points in the train set: 79687, number of used features: 47
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.213423 -> initscore=-1.304417
[LightGBM] [Info] Start training from score -1.304417


LightGBM, Decision Tree, SGD and Neural Network are trained on dataset.


In [24]:
result_analysis = model_runner.evaluate_models()
result_analysis

{'Decision Tree': {'Accuracy': 0.87,
  'Precision': 0.69,
  'Recall': 0.71,
  'F1 Score': 0.7},
 'LightGBM': {'Accuracy': 0.89,
  'Precision': 0.83,
  'Recall': 0.59,
  'F1 Score': 0.69},
 'SGD Classifier': {'Accuracy': 0.73,
  'Precision': 0.37,
  'Recall': 0.34,
  'F1 Score': 0.35},
 'XGBoost': {'Accuracy': 0.9,
  'Precision': 0.83,
  'Recall': 0.67,
  'F1 Score': 0.74},
 'Neural Network': {'Accuracy': 0.81,
  'Precision': 0.71,
  'Recall': 0.18,
  'F1 Score': 0.28}}

In [26]:
result_df = pd.DataFrame(result_analysis)
result_df.to_csv('rejecction_results.csv',index=False)

In [28]:
top_model_features = model_runner.get_xgboost_feature_importance()

features_df = pd.DataFrame({"Column_name":X_test_encoded.columns,"Feature_importance":top_model_features}).sort_values(
    by=['Feature_importance'],ascending=False)
features_df

,Column_name,Feature_importance
29,PURCHASER_CODE,0.093649
27,DEPARTMENT_TYPE,0.064988
6,VAT_PERCENTAGE,0.063160
28,TREATMENT_TYPE,0.058410
14,PROVIDER_DEPARTMENT_CODE,0.053595
12,DISCOUNT_PERCENTAGE,0.044982
10,UNIT_PRICE_STOCKED_UOM,0.031178
2,LINE_CLAIMED_AMOUNT_SAR,0.030961
37,CombinedText5,0.030481
38,CombinedText6,0.030072


In [29]:
features_df.to_csv('Features_importance.csv',index=False)

## Rejection Reason Notes Codes Analysis


In [5]:
from src.utilities_rejection import RejectionReasonLabeling
from src.data_local import MergedDataPreprocessing
from cashe import get_cashed_input,read_cashed_original

#df = get_cashed_input()
df = pd.read_csv('data/HJH/12-06-2024/df.csv')

C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_12424\3523704105.py:6: DtypeWarning: Columns (26,39,41,49,51) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/HJH/12-06-2024/df.csv')


In [6]:
df['OUTCOME'].value_counts()

OUTCOME
APPROVED    107638
REJECTED     85261
PARTIAL      14306
Name: count, dtype: int64

In [7]:
label_technical = RejectionReasonLabeling(df)
label_technical.recoginze_label()

In [43]:
#df_original = read_cashed_original()
#label_technical.recognize_service(df_original)

In [8]:
df

,CREATED_BY_USER,CREATION_DATE,VISIT_ID,NET_WITH_VAT,SERVICE_DESCRIPTION,QTY,LINE_CLAIMED_AMOUNT,LINE_CLAIMED_AMOUNT_SAR,CO_PAY,CO_INSURANCE,...,NEW_BORN,PATIENT_IDENTITY_TYPE,BILLABLE_PERIOD_START,BILLABLE_PERIOD_END,BATCH_PERIOD_START,BATCH_PERIOD_END,Episode_Key,ICD10,Diagnosis_Key,NPHIES_CODE
0,10335,2024-01-01 09:58:15,194131,575.00,Routine preoperative anaes assessment,1.0,500.00,500.00,0.0,500.00,...,N,H,2023-12-07 10:29:10,2023-12-09 09:52:10,2023-12-01,2023-12-10,1_1335736-8,O82,1_1111231440,None
1,10335,2024-01-01 09:58:15,194131,312.80,Cross Matching,1.0,340.00,340.00,0.0,340.00,...,N,H,2023-12-07 10:29:10,2023-12-09 09:52:10,2023-12-01,2023-12-10,1_1335736-8,O82,1_1111231440,None
2,10335,2024-01-01 09:58:15,194131,11.38,TRANDATE INJ. 100 MG.-AMP.20ML,1.0,11.98,11.98,0.0,11.98,...,N,H,2023-12-07 10:29:10,2023-12-09 09:52:10,2023-12-01,2023-12-10,1_1335736-8,O82,1_1111231440,None
3,10335,2024-01-01 09:58:15,194131,15346.75,Elective classical caesarean section,1.0,15700.00,15700.00,0.0,15700.00,...,N,H,2023-12-07 10:29:10,2023-12-09 09:52:10,2023-12-01,2023-12-10,1_1335736-8,O82,1_1111231440,None
4,10335,2024-01-01 10:00:43,194132,0.88,"Hypodermic needle, single-use",1.0,1.10,1.10,0.0,1.10,...,N,H,2023-12-07 14:09:32,2023-12-09 09:52:28,2023-12-01,2023-12-10,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478139,9638,2024-05-31 18:36:40,252592,113.84,VANIN 500 MG 10 VIAL,4.0,122.40,122.40,0.0,122.40,...,N,H,2024-04-16 07:45:09,2024-05-05 22:07:21,2024-05-01,2024-05-15,NaN,NaN,NaN,None
478140,9638,2024-05-31 18:36:40,252592,56.92,VANIN 500 MG 10 VIAL,2.0,61.20,61.20,0.0,61.20,...,N,H,2024-04-16 07:45:09,2024-05-05 22:07:21,2024-05-01,2024-05-15,NaN,NaN,NaN,None
478141,9638,2024-05-31 18:36:40,252592,168.94,Room and Board ICU excluding consumables,1.0,226.00,226.00,0.0,226.00,...,N,H,2024-04-16 07:45:09,2024-05-05 22:07:21,2024-05-01,2024-05-15,NaN,NaN,NaN,None
478142,9638,2024-05-31 18:36:40,252592,2.08,TAZMERON 15 MG 30 TABLET,1.0,2.24,2.24,0.0,2.24,...,N,H,2024-04-16 07:45:09,2024-05-05 22:07:21,2024-05-01,2024-05-15,NaN,NaN,NaN,None


In [9]:
df.groupby('NPHIES_CODE')['LINE_CLAIMED_AMOUNT'].sum().reset_index().sort_values(['LINE_CLAIMED_AMOUNT'],ascending=False)

,NPHIES_CODE,LINE_CLAIMED_AMOUNT
20,CV-1-1,7112957.40
31,MN-1-1,3791351.04
36,SE-1-6,3086728.68
19,BE-1-7,2507428.10
16,BE-1-4,1973048.38
12,BE-1-1,1865733.43
15,BE-1-3,1779970.90
18,BE-1-6,1295824.43
38,SE-1-8,688381.38
17,BE-1-5,535632.13


In [91]:
df[df['NPHIES_CODE']=='SE-1-9'][['PURCHASER_CODE','CONTRACT_NO','PAYER_LICENSE']].drop_duplicates()

,PURCHASER_CODE,CONTRACT_NO,PAYER_LICENSE
11985,2126,74298,7001519912
12030,2126,74302,7001519912
12126,2126,116960,7001519912
12138,2126,74301,7001519912
12825,607,85088,7001519912
12910,607,118918,7001519912
12926,607,60747,7001519912
13125,607,96770,7001519912
13168,607,122102,7001519912
13192,607,115439,7001519912


In [95]:
df[df.QTY.isin([1.0,2.0,3.0])].QTY.value_counts().sum() / len(df)

0.8049165105072948

In [96]:
len(df)

478144

In [63]:
df.groupby('NPHIES_CODE')['LINE_CLAIMED_AMOUNT'].sum().reset_index().sort_values(['LINE_CLAIMED_AMOUNT'],ascending=False).\
    to_csv('data/Extras/2024_08_02/helper/a.csv',index=True)

In [69]:
df[df['NPHIES_CODE']=='CV-1-1'][['CONTRACT_NO','HIS_INSURANCE_CODE']]

,CONTRACT_NO,HIS_INSURANCE_CODE
398360,74294,2134
398364,117860,2134
398363,117860,2134
398365,117860,2134
398361,117860,2134
...,...,...
387587,74304,2134
387926,74304,2134
388312,72725,2134
388311,72725,2134


In [78]:
df[df['NPHIES_CODE']=='CV-3-2'].SERVICE_DESCRIPTION.value_counts()

SERVICE_DESCRIPTION
Unlisted Code                                                                                                                    202
Adhesive restoration(eg. compomer, composite, glass ionomer, polymer glass, resin composite), each additional surface, direct    139
Uric Acid Level, Blood                                                                                                            16
GHBA1C                                                                                                                            13
TSH                                                                                                                                7
Syringe-loaded injector, manual, professional                                                                                      4
Computerised tomography of chest                                                                                                   4
IV admin of pharmac agt oth   unsp agent         

In [55]:
df[df['NPHIES_CODE']=='AD-2-4']

,CREATED_BY_USER,CREATION_DATE,VISIT_ID,NET_WITH_VAT,SERVICE_DESCRIPTION,QTY,LINE_CLAIMED_AMOUNT,LINE_CLAIMED_AMOUNT_SAR,CO_PAY,CO_INSURANCE,...,NEW_BORN,PATIENT_IDENTITY_TYPE,BILLABLE_PERIOD_START,BILLABLE_PERIOD_END,BATCH_PERIOD_START,BATCH_PERIOD_END,Episode_Key,ICD10,Diagnosis_Key,NPHIES_CODE
420945,10708,2024-01-02 18:09:30,194912,83.16,"Dressing of wound, small",1.0,99.00,99.00,14.85,84.15,...,N,H,2023-12-12 00:00:00,2023-12-26 00:00:00,2023-12-01,2023-12-31,1_783381-39,T30,1_110237929,AD-2-4
477555,10658,2024-01-02 18:15:47,194944,14.96,PANADOL EXTEND 665MG F.C. TABLETS,24.0,15.75,15.75,2.99,12.76,...,N,H,2023-12-26 00:00:00,2024-01-09 00:00:00,2023-12-01,2023-12-31,1_878295-9,J06.9,1_10164354,AD-2-4
408941,10658,2024-01-02 18:15:47,194944,7.17,DECOZAL NASAL SPRAY,1.0,7.55,7.55,1.43,6.12,...,N,H,2023-12-26 00:00:00,2024-01-09 00:00:00,2023-12-01,2023-12-31,1_878295-9,J06.9,1_10164354,AD-2-4
408942,10658,2024-01-02 18:15:47,194944,13.01,PARACETOL 10 MG/ML 100 ML BOTTLE (PSI),1.0,13.70,13.70,2.60,11.10,...,N,H,2023-12-26 00:00:00,2024-01-09 00:00:00,2023-12-01,2023-12-31,1_878295-9,J06.9,1_10164354,AD-2-4
408943,10658,2024-01-02 18:15:47,194944,18.43,LEVOZAL 5 MG FILM COATED TABLET,20.0,19.40,19.40,3.69,15.71,...,N,H,2023-12-26 00:00:00,2024-01-09 00:00:00,2023-12-01,2023-12-31,1_878295-9,J06.9,1_10164354,AD-2-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377361,10335,2024-04-25 17:18:00,237570,175.95,Load drug delv device oth unsp agt,1.0,204.00,204.00,0.00,204.00,...,N,BN,2024-03-02 00:18:18,2024-03-18 16:04:53,2024-03-01,2024-03-31,1_1360604-2,R57.8,1_1111232265,AD-2-4
377360,10335,2024-04-25 17:18:00,237570,276.00,Intrathcl admin pharmac agt oth unsp,4.0,320.00,320.00,0.00,320.00,...,N,BN,2024-03-02 00:18:18,2024-03-18 16:04:53,2024-03-01,2024-03-31,1_1360604-2,R57.8,1_1111232265,AD-2-4
378157,10335,2024-04-25 17:18:00,237570,430.39,ABG,1.0,499.00,499.00,0.00,499.00,...,N,BN,2024-03-02 00:18:18,2024-03-18 16:04:53,2024-03-01,2024-03-31,1_1360604-2,R57.8,1_1111232265,AD-2-4
378159,10335,2024-04-25 17:18:00,237570,263.58,Blood sugar by glucometer,4.0,305.60,305.60,0.00,305.60,...,N,BN,2024-03-02 00:18:18,2024-03-18 16:04:53,2024-03-01,2024-03-31,1_1360604-2,R57.8,1_1111232265,AD-2-4


In [64]:
df.columns

Index(['CREATED_BY_USER', 'CREATION_DATE', 'VISIT_ID', 'NET_WITH_VAT',
       'SERVICE_DESCRIPTION', 'QTY', 'LINE_CLAIMED_AMOUNT',
       'LINE_CLAIMED_AMOUNT_SAR', 'CO_PAY', 'CO_INSURANCE',
       'LINE_ITEM_DISCOUNT', 'NET_AMOUNT', 'NET_VAT_AMOUNT',
       'PATIENT_VAT_AMOUNT', 'VAT_PERCENTAGE', 'TREATMENT_TYPE_INDICATOR',
       'SERVICE_TYPE', 'DURATION', 'QTY_STOCKED_UOM', 'UNIT_PRICE',
       'OASIS_IOS_DESCRIPTION', 'SEQUENCE_NO', 'UNIT_PRICE_STOCKED_UOM',
       'UNIT_PRICE_NET', 'DISCOUNT_PERCENTAGE', 'NOTES', 'OUTCOME',
       'APPROVED_QUNATITY', 'REQUEST_DATE', 'VISIT_NO', 'VISIT_DATE',
       'EMERGENCY_INDICATOR', 'PROVIDER_DEPARTMENT_CODE',
       'PROVIDER_DEPARTMENT', 'DOCTOR_SPECIALTY_CODE', 'DOCTOR_CODE',
       'PATIENT_NO', 'EPISODE_NO', 'PATIENT_ID', 'MEMBERSHIP_NO',
       'PATIENT_ID_TYPE', 'PATIENT_NATIONAL_IQAMA_ID', 'PATIENT_DOB',
       'PATIENT_AGE', 'UNIT_OF_AGE', 'PATIENT_NATIONALITY',
       'PATIENT_MARITAL_STATUS', 'PATIENT_GENDER', 'PATIENT_MOBILE_NO'

In [57]:
#df.DUPLICATED_SERVICE.value_counts()

In [ ]:
## CV-x-x problems are related to the provider contract mainly, which can't be a part of our predictive analytics

In [4]:
## MN-1-1 Service is not clinically justified based on clinical practice guideline (Service vs Diagnosis)
df[df['NPHIES_CODE'] == 'MN-1-1'][['SERVICE_DESCRIPTION', 'ICD10']]
## DERMATIN SOLUTION with ICD10 (H60) indicating Otitis externa ( Drug is incompatible with icd10 ).
## DERMATIN for foot itches, and otitis externa could be treated with hydrocortisone 3 drops to the affected ear daily they are two different things.

,SERVICE_DESCRIPTION,ICD10
397397,DERMATIN SOLUTION 1%,H60
397398,DEXAFLOX OPHTHALMIC SUSPENSION,H60
397468,OPTIDEX-T STERILE OPHTALMIC SUSP,H00.1
397606,PARACETOL 10 MG/ML 100 ML BOTTLE (PSI),H66.9
397541,PANADOL ADVANCE 500MG FILM COATED TABLET,H66.9
...,...,...
388372,RIAZOLE,Z39.00
388376,CRP,P51.9
388377,Automated complete Blood cell and differentia...,P51.9
388378,AVOBAN 2% W-W OINTMENT,P51.9


In [5]:
## SE-1-6: Investigation result is inadequate or missing
df[df['NPHIES_CODE'] == 'SE-1-6'][
    ['SERVICE_DESCRIPTION', 'PATIENT_NO', 'PATIENT_ID', 'MEMBERSHIP_NO', 'PATIENT_ID_TYPE', 'PATIENT_NATIONAL_IQAMA_ID',
     'PATIENT_DOB', 'PATIENT_NATIONALITY', 'PATIENT_MARITAL_STATUS', 'PATIENT_GENDER', 'PATIENT_MOBILE_NO',
     'PATIENT_BADGE_NO', 'HIS_INSURANCE_CODE', 'POLICY_HOLDER_NO']]
## SE-1-8,9: Treatment plan is inadequate or missing, Quantity of service/procedure is inappropriate or missing

,SERVICE_DESCRIPTION,PATIENT_NO,PATIENT_ID,MEMBERSHIP_NO,PATIENT_ID_TYPE,PATIENT_NATIONAL_IQAMA_ID,PATIENT_DOB,PATIENT_NATIONALITY,PATIENT_MARITAL_STATUS,PATIENT_GENDER,PATIENT_MOBILE_NO,PATIENT_BADGE_NO,HIS_INSURANCE_CODE,POLICY_HOLDER_NO
400859,"Radiography of teeth, full mouth",876090,1020440,1079611,1.0,1195216146,1996-10-16,SAUDI ARABIAN,S,F,551444436,1079611,275,1116797745
401581,"Radiography of teeth, full mouth",1182339,1345812,5291507,1.0,1084294865,1989-02-27,SAUDI ARABIAN,M,F,566102603,5291507,275,GRH/15163579
402709,"Radiography of teeth, full mouth",124165,51877,1417318,1.0,1003238548,1948-05-09,SAUDI ARABIAN,M,M,555644665,1417318,275,14888432
400522,"Radiography of teeth, full mouth",1014161,1167940,635153,2.0,2037303993,1983-10-27,YEMENI,M,M,500211499,635153,275,GRH/15218233-0
400537,"Radiography of teeth, full mouth",1044181,1199407,635156,2.0,2223614682,2005-07-17,YEMENI,S,M,500211499,635156,275,2037303993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325911,LAMICTAL 100 MG 30 TAB,137825,176890,54925667,1.0,1009343847,1973-07-30,SAUDI ARABIAN,M,M,503627060,54925667,2622,2230000326
325914,OLFEN PATCHES,137825,176890,54925667,1.0,1009343847,1973-07-30,SAUDI ARABIAN,M,M,503627060,54925667,2622,2230000326
325912,NEXIUM 20MG TAB BOX/28,137825,176890,54925667,1.0,1009343847,1973-07-30,SAUDI ARABIAN,M,M,503627060,54925667,2622,2230000326
339352,M-mode 2D real time u/s of heart,1196636,1361172,6755132,2.0,2008467603,1955-07-01,SUDANESE,M,M,507439462,6755132,183,185182033


In [6]:
df[df['NPHIES_CODE'].isin(['SE-1-9', 'SE-1-8'])][
    ['QTY', 'LINE_CLAIMED_AMOUNT', 'LINE_CLAIMED_AMOUNT_SAR', 'CO_PAY', 'CO_INSURANCE', 'LINE_ITEM_DISCOUNT',
     'NET_AMOUNT', 'NET_VAT_AMOUNT']]
## some items quantities are way too many

,QTY,LINE_CLAIMED_AMOUNT,LINE_CLAIMED_AMOUNT_SAR,CO_PAY,CO_INSURANCE,LINE_ITEM_DISCOUNT,NET_AMOUNT,NET_VAT_AMOUNT
404193,1.0,750.0,750.0,0.0,750.0,0.00,750.00,112.50
404200,1.0,135.0,135.0,0.0,135.0,33.75,101.25,15.19
403452,2.0,16.0,16.0,0.0,16.0,0.80,15.20,0.00
403475,2.0,1160.0,1160.0,0.0,1160.0,290.00,870.00,130.50
404226,1.0,5.4,5.4,0.0,5.4,1.35,4.05,0.00
...,...,...,...,...,...,...,...,...
229082,2.0,3380.0,3380.0,0.0,3380.0,0.00,3380.00,507.00
229083,1.0,810.0,810.0,0.0,810.0,202.50,607.50,91.13
229446,2.0,2070.0,2070.0,0.0,2070.0,0.00,2070.00,310.50
229444,1.0,1820.0,1820.0,0.0,1820.0,0.00,1820.00,273.00


In [97]:
df[df['NPHIES_CODE'].isin(['SE-1-8'])][ ['QTY', 'LINE_CLAIMED_AMOUNT', 'LINE_CLAIMED_AMOUNT_SAR', 'CO_PAY', 'CO_INSURANCE', 'LINE_ITEM_DISCOUNT',  'NET_AMOUNT', 'NET_VAT_AMOUNT']]

,QTY,LINE_CLAIMED_AMOUNT,LINE_CLAIMED_AMOUNT_SAR,CO_PAY,CO_INSURANCE,LINE_ITEM_DISCOUNT,NET_AMOUNT,NET_VAT_AMOUNT
71507,1.0,12362.51,12362.51,0.0,12362.51,1854.38,10508.13,1576.22
72985,1.0,104.81,104.81,0.0,104.81,31.44,73.37,11.01
72986,1.0,446.13,446.13,0.0,446.13,133.84,312.29,46.84
72989,1.0,365.50,365.50,0.0,365.50,109.65,255.85,38.38
72990,1.0,104.81,104.81,0.0,104.81,31.44,73.37,11.01
...,...,...,...,...,...,...,...,...
242591,1.0,154.71,154.71,0.0,154.71,54.15,100.56,15.08
242592,1.0,154.71,154.71,0.0,154.71,54.15,100.56,15.08
242593,1.0,276.98,276.98,0.0,276.98,96.94,180.04,27.01
244786,6.0,3762.48,3762.48,0.0,3762.48,1128.74,2633.74,395.06


In [98]:
df[df['NPHIES_CODE'].isin(['SE-1-8'])][ ['QTY', 'LINE_CLAIMED_AMOUNT', 'LINE_CLAIMED_AMOUNT_SAR', 'CO_PAY', 'CO_INSURANCE', 'LINE_ITEM_DISCOUNT',  'NET_AMOUNT', 'NET_VAT_AMOUNT']].QTY.value_counts()

QTY
1.0     1124
2.0      211
3.0       78
4.0       67
5.0       42
6.0       32
10.0      28
8.0       12
20.0       7
14.0       1
12.0       1
30.0       1
15.0       1
7.0        1
9.0        1
40.0       1
Name: count, dtype: int64

In [7]:
## BE-1-1 : Co-pay was not collected from member
df[df['NPHIES_CODE'].isin(['BE-1-1'])][['QTY', 'CO_PAY', 'CO_INSURANCE', 'LINE_ITEM_DISCOUNT', 'NET_AMOUNT']]
## Most of CO_PAY values are zeros. (MOST)

,QTY,CO_PAY,CO_INSURANCE,LINE_ITEM_DISCOUNT,NET_AMOUNT
407021,7.0,13.02,108.29,6.07,102.22
477325,14.0,8.47,36.13,2.23,33.90
407392,1.0,0.00,180.00,0.00,180.00
407384,1.0,0.00,269.00,67.25,201.75
407399,90.0,0.00,6.35,0.32,6.03
...,...,...,...,...,...
387799,1.0,16.30,100.15,34.94,65.21
388099,1.0,3.76,23.11,8.06,15.05
388366,1.0,8.15,50.08,17.47,32.61
388367,1.0,8.15,50.08,17.47,32.61


In [99]:
df[df['NPHIES_CODE'].isin(['BE-1-1'])].CO_PAY.value_counts()

CO_PAY
0.00     4505
24.00     544
16.30     482
14.67     296
8.15      230
         ... 
17.06       1
35.69       1
17.95       1
46.65       1
9.36        1
Name: count, Length: 674, dtype: int64

In [100]:
## portion of zeros
4500 / 7200

0.625

In [102]:
df[df['NPHIES_CODE'] !='BE-1-1'].CO_PAY.value_counts()

CO_PAY
0.00      300638
24.00       7189
20.00       4336
25.00       4145
34.00       4056
           ...  
26.33          1
31.43          1
389.12         1
57.58          1
45.17          1
Name: count, Length: 4104, dtype: int64

In [103]:
300638 / len(df)

0.6287603734439834

In [8]:
## BE-1-3: provider and the payer disagreement
df2 = df[df['NPHIES_CODE'].isin(['BE-1-3'])][['PROVIDER_DEPARTMENT', 'HIS_INSURANCE_CODE']]
df2[df2['HIS_INSURANCE_CODE'] == 1494].PROVIDER_DEPARTMENT.unique()

array(['INTERNAL MEDICINE - الباطنة', 'OPHTHALMIC SERVICES  - العيون',
       'E.N.T. SERVICES - انف واذن وحنجرة',
       'EMERGENCY ROOM SERVICES - طوارى', 'PAEDIATRIC SERVICES - الاطفال',
       'OBSTERTRIC/GYNEACOLOGIC SERVIC - نساء و ولادة',
       'ORTHOPAEDIC SERVICE - العظام', 'NEUROLOGY SERVICES مخ واعصاب',
       'CARDIOLOGY SERVICES - القلب', 'DENTAL SERVICES - اسنان',
       'NEURO SURGERY - جراحة مخ واعصاب', 'PULMONARY SERVICES - الصدرية',
       'UROLOGY SERVICES -المسالك',
       'GENERAL SURGERY SERVICES - جراحة عامة',
       'INTENSIVE CARE SERVICES - العناية المركزة',
       'DERMATOLOGY & VENEREOLOGY SERV - جلدية'], dtype=object)

In [9]:
df_2 = df[df['NPHIES_CODE'] != 'BE-1-3'][['PROVIDER_DEPARTMENT', 'HIS_INSURANCE_CODE']]
df_2[df_2['HIS_INSURANCE_CODE'] == 1494].PROVIDER_DEPARTMENT.unique()
## Notice that the same set of departments exists twice

array(['E.N.T. SERVICES - انف واذن وحنجرة',
       'ORTHOPAEDIC SERVICE - العظام', 'NEUROLOGY SERVICES مخ واعصاب',
       'PULMONARY SERVICES - الصدرية', 'OPHTHALMIC SERVICES  - العيون',
       'OBSTERTRIC/GYNEACOLOGIC SERVIC - نساء و ولادة',
       'INTERNAL MEDICINE - الباطنة', 'EMERGENCY ROOM SERVICES - طوارى',
       'DENTAL SERVICES - اسنان',
       'DERMATOLOGY & VENEREOLOGY SERV - جلدية',
       'PAEDIATRIC SERVICES - الاطفال', 'UROLOGY SERVICES -المسالك',
       'CARDIOLOGY SERVICES - القلب', 'NEURO SURGERY - جراحة مخ واعصاب',
       'PLASTIC SURGERY - التجميل',
       'GENERAL SURGERY SERVICES - جراحة عامة',
       'INTENSIVE CARE SERVICES - العناية المركزة',
       'RADIOGRAPHIC INVESTIGATIONS - الأشعة', 'SCREENING DEPARTMENT',
       'MAXILLOFACIAL SURGERY'], dtype=object)

In [10]:
df.columns
## BE-1-4: Preauthorization is required and was not obtained (Can't be modeled)
## BE-1-6: Calculation discrepancy from contract (Can't be modeled)
## BE-1-7: Incorrect billing regime An inpatient claim is submitted as Fee for Service when it should be billed as DRG.

Index(['CREATED_BY_USER', 'CREATION_DATE', 'VISIT_ID', 'NET_WITH_VAT',
       'SERVICE_DESCRIPTION', 'QTY', 'LINE_CLAIMED_AMOUNT',
       'LINE_CLAIMED_AMOUNT_SAR', 'CO_PAY', 'CO_INSURANCE',
       'LINE_ITEM_DISCOUNT', 'NET_AMOUNT', 'NET_VAT_AMOUNT',
       'PATIENT_VAT_AMOUNT', 'VAT_PERCENTAGE', 'TREATMENT_TYPE_INDICATOR',
       'SERVICE_TYPE', 'DURATION', 'QTY_STOCKED_UOM', 'UNIT_PRICE',
       'OASIS_IOS_DESCRIPTION', 'SEQUENCE_NO', 'UNIT_PRICE_STOCKED_UOM',
       'UNIT_PRICE_NET', 'DISCOUNT_PERCENTAGE', 'NOTES', 'OUTCOME',
       'APPROVED_QUNATITY', 'REQUEST_DATE', 'VISIT_NO', 'VISIT_DATE',
       'EMERGENCY_INDICATOR', 'PROVIDER_DEPARTMENT_CODE',
       'PROVIDER_DEPARTMENT', 'DOCTOR_SPECIALTY_CODE', 'DOCTOR_CODE',
       'PATIENT_NO', 'EPISODE_NO', 'PATIENT_ID', 'MEMBERSHIP_NO',
       'PATIENT_ID_TYPE', 'PATIENT_NATIONAL_IQAMA_ID', 'PATIENT_DOB',
       'PATIENT_AGE', 'UNIT_OF_AGE', 'PATIENT_NATIONALITY',
       'PATIENT_MARITAL_STATUS', 'PATIENT_GENDER', 'PATIENT_MOBILE_NO'

In [11]:
df[df['NPHIES_CODE'] != 'BE-1-7'][['PATIENT_ID_TYPE', 'TOTAL_DISCOUNT', 'TOTAL_NET_AMOUNT', 'CLAIM_TYPE']]
## AD-2-4, AD-2-6 : duplicated service or procedure in the same claim, missing the date of claim, and / or claim is submitted after 5 months of the date of service

,PATIENT_ID_TYPE,TOTAL_DISCOUNT,TOTAL_NET_AMOUNT,CLAIM_TYPE
397551,1.0,2763.6,15488.38,I
397548,1.0,2763.6,15488.38,I
397549,1.0,2763.6,15488.38,I
397550,1.0,2763.6,15488.38,I
397559,1.0,298.6,2352.96,I
...,...,...,...,...
388637,1.0,362.2,893.03,O
388638,1.0,362.2,893.03,O
388639,1.0,362.2,893.03,O
388640,1.0,362.2,893.03,O


In [12]:
df[df['NPHIES_CODE'].isin(["AD-2-5", "AD-2-6"])][
    ['CREATION_DATE', 'BATCH_PERIOD_START', 'BATCH_PERIOD_END', 'SERVICE_DESCRIPTION']]

,CREATION_DATE,BATCH_PERIOD_START,BATCH_PERIOD_END,SERVICE_DESCRIPTION
419923,2024-01-02 16:25:59,2023-12-01,2023-12-31,Dengue IgM/IgG
419924,2024-01-02 16:25:59,2023-12-01,2023-12-31,FARCOLIN 0.5% RESPIRATOR SOLN.
419925,2024-01-02 16:25:59,2023-12-01,2023-12-31,OSELTA 75MG CAPSULES
419926,2024-01-02 16:25:59,2023-12-01,2023-12-31,"Nebulizing system, heated"
419927,2024-01-02 16:25:59,2023-12-01,2023-12-31,Unsp admin of pharmac agent electrolyte
...,...,...,...,...
339407,2024-04-14 16:39:49,2024-03-01,2024-03-31,PARACETOL 10 MG/ML 100 ML BOTTLE (PSI)
340182,2024-04-14 16:46:09,2024-03-01,2024-03-31,Unlisted Code
340183,2024-04-14 16:46:10,2024-03-01,2024-03-31,Quantitative Troponin (Protein) Analysis
396630,2024-04-14 16:46:11,2024-03-01,2024-03-31,"Radiography of chest, 1 view"


In [13]:
df[df['NPHIES_CODE'] == "AD-2-5"][['CREATION_DATE', 'BATCH_PERIOD_START', 'BATCH_PERIOD_END', 'SERVICE_DESCRIPTION']]

,CREATION_DATE,BATCH_PERIOD_START,BATCH_PERIOD_END,SERVICE_DESCRIPTION
420802,2024-01-02 19:50:02,2023-12-01,2023-12-31,FLUCA OPTHALMIC SUSP.
420803,2024-01-02 19:50:02,2023-12-01,2023-12-31,Consultant Consultation
420801,2024-01-02 19:50:02,2023-12-01,2023-12-31,TOBRADEX EYE OINT
414051,2024-01-03 17:40:06,2023-12-01,2023-12-31,SOCLAV 1 G/125 MG 16 SACHET
414050,2024-01-03 17:40:06,2023-12-01,2023-12-31,SOLPADEINE SOLUBLE TABLET
...,...,...,...,...
291711,2024-04-06 05:54:25,2024-03-01,2024-03-31,SALURIN 20MG-2ML AMPOULES
291712,2024-04-06 05:54:25,2024-03-01,2024-03-31,SALURIN 20MG-2ML AMPOULES
291714,2024-04-06 05:54:25,2024-03-01,2024-03-31,EPINOR 4 MG / 4 ML 5 AMP
291706,2024-04-06 05:54:25,2024-03-01,2024-03-31,Unlisted Procedure Code


In [14]:
df[df['NPHIES_CODE'] == "AD-2-6"][['CREATION_DATE', 'BATCH_PERIOD_START', 'BATCH_PERIOD_END', 'SERVICE_DESCRIPTION']]
df.CREATION_DATE.sort_values().iloc[-1]

Timestamp('2024-04-30 12:13:09')

In [15]:
df[~df['NPHIES_CODE'].isin(["AD-2-4", "AD-2-5", "AD-2-6"])][
    ['CREATION_DATE', 'BATCH_PERIOD_START', 'BATCH_PERIOD_END', 'SERVICE_DESCRIPTION']]

,CREATION_DATE,BATCH_PERIOD_START,BATCH_PERIOD_END,SERVICE_DESCRIPTION
397551,2024-01-01 09:58:15,2023-12-01,2023-12-10,Routine preoperative anaes assessment
397548,2024-01-01 09:58:15,2023-12-01,2023-12-10,Cross Matching
397549,2024-01-01 09:58:15,2023-12-01,2023-12-10,TRANDATE INJ. 100 MG.-AMP.20ML
397550,2024-01-01 09:58:15,2023-12-01,2023-12-10,Elective classical caesarean section
397559,2024-01-01 10:00:43,2023-12-01,2023-12-10,"Hypodermic needle, single-use"
...,...,...,...,...
388637,2024-04-30 12:13:09,2024-04-01,2024-04-10,Peripheral intravenous cannula
388638,2024-04-30 12:13:09,2024-04-01,2024-04-10,Ultrasound of pelvis
388639,2024-04-30 12:13:09,2024-04-01,2024-04-10,Urinalysis Test
388640,2024-04-30 12:13:09,2024-04-01,2024-04-10,PANADREX 500 MG 24 TAB


In [ ]:
df['OUTCOME'].value_counts()